In [2]:
import pandas as pd
import numpy as np
import re
import copy
import dtale
import os
#pd.set_option("Max_columns", None)

### Table 3. Descriptive Statistics- IVs

In [3]:
dt_master = pd.read_excel("data/Minerva_Clustering_20220209_MARupdated_Vdemadded.xlsx")
dt_pitf = pd.read_excel("data/PITF_2yrpeace_2yrlag.xlsx")

In [28]:
dt_master[(dt_master.year<=2012) & (dt_master.year>=1960)][dt_master.columns[:-2]].dropna().describe()

,year,polity_squared,PopYouthBulgeBy15_new,gdppc_ln,nAC_perc,imr_annual_mean_centered,sum_PDIS_EDIS
count,6260.000000,6260.000000,6260.000000,6260.000000,6260.000000,6260.000000,6260.000000
mean,1989.027157,55.319649,41.344049,8.674893,0.256262,3.797022,7.713738
std,14.606777,31.906626,8.924901,1.291389,0.303323,42.457701,8.761244
min,1960.000000,0.000000,17.885186,2.041869,0.000000,-84.436250,0.000000
25%,1977.000000,36.000000,33.799559,7.675125,0.000000,-28.580936,0.000000
50%,1990.000000,49.000000,44.820292,8.643223,0.200000,-3.658023,6.000000
75%,2002.000000,81.000000,48.056279,9.656438,0.428571,35.011667,12.000000
max,2012.000000,100.000000,56.409996,12.367041,1.000000,136.063750,57.000000


In [16]:
dt_pitf[dt_pitf.columns[:-4]].dropna().describe()

,year,polity_squared,PopYouthBulgeBy15_new,gdppc_ln,nAC_perc,imr_annual_mean_centered,sum_PDIS_EDIS
count,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000
mean,1989.368421,40.486842,44.630487,8.098927,0.373630,22.507192,12.434211
std,13.439089,26.021270,6.877194,1.199196,0.315258,40.152124,10.742235
min,1965.000000,0.000000,24.309952,2.367905,0.000000,-72.982143,0.000000
25%,1978.000000,22.750000,43.514676,7.291633,0.000000,-13.981012,5.000000
50%,1990.000000,49.000000,46.865368,8.128100,0.366667,22.561779,8.000000
75%,2000.000000,49.000000,48.553502,9.012391,0.578571,54.472120,19.000000
max,2014.000000,100.000000,53.236774,10.253272,1.000000,109.168627,40.000000


### Table 4 & 5 & 6. Descriptive Statistics- Sensitivity Analysis

In [31]:
cluster_char = dict()
for file in os.listdir("output"):
    if "3clust_" in file:
        method = file[7:-5]
        f_output = pd.ExcelFile(f"output/{file}")
        dt_agnes = f_output.parse(sheet_name="3clust_agnes")
        dt_pam = f_output.parse(sheet_name="3clust_PAM")
        cluster_char[f"{method}_agnes"] = dt_agnes
        cluster_char[f"{method}_pam"] = dt_pam
cluster_char.keys()

dict_keys(['eucl_pitf_agnes', 'eucl_pitf_pam', 'eucl_pitf_vdem_agnes', 'eucl_pitf_vdem_pam', 'eucl_prio_agnes', 'eucl_prio_pam', 'eucl_prio_vdem_agnes', 'eucl_prio_vdem_pam', 'gower_pitf_agnes', 'gower_pitf_pam', 'gower_pitf_vdem_agnes', 'gower_pitf_vdem_pam', 'gower_prio_agnes', 'gower_prio_pam', 'gower_prio_vdem_agnes', 'gower_prio_vdem_pam'])

In [32]:
c_var = [
    'polity_squared', 
          'PopYouthBulgeBy15_new', 
          'gdppc_ln',
          'nAC_perc',
          'imr_annual_mean_centered',
          'sum_PDIS_EDIS'
]
c_var_mm = [*[f"{c}_mean" for c in c_var], *[f"{c}_median" for c in c_var], *[f"{c}_scale_mean" for c in c_var]]
dt_summary = []
for k in cluster_char:
    dt = copy.deepcopy(cluster_char[k])
    cl = [*[i for i in dt.columns if i.endswith("_cluster")],*["n"], *c_var_mm]
    dt = dt[cl]
    dt.insert(0, "method", k)
    if k == 
    dt_summary.append(dt)
dt_summary = pd.concat(dt_summary)
#
dt_summary.agnes_cluster = dt_summary.agnes_cluster.fillna(dt_summary.pam_cluster)
dt_summary.round(2).to_excel("output/3clusters_summary.xlsx", index=False)

In [79]:
dt_summary_fix = pd.read_excel("output/3clusters_summary clusterfix 20220210.xlsx")
c_mean = [c for c in dt_summary_fix.columns if c.endswith("_mean") and "_scale_" not in c ]
c_median = [c for c in dt_summary_fix.columns if c.endswith("_median") and "_scale_" not in c ]
dt_mean = dt_summary_fix[[*['method', 'Cluster', 'n'],*c_mean]].copy()
dt_median = dt_summary_fix[[*['method', 'Cluster', 'n'],*c_median]].copy()
dt_mean.columns = [c[:-5] if c.endswith("_mean") else c for c in dt_mean.columns]
dt_median.columns = [c.replace("_median", "") for c in dt_median.columns]
dt_mean.method = dt_mean.method.str.replace("_", "+").str.upper()
dt_median.method = dt_median.method.str.replace("_", "+").str.upper()
##
clu_map = {
    1: "Anocratic - Younger - Less Developed",
    2:"Older - Wealthier - Moderate Discrimination",
    3:"Higher Discrimination - Worse Neighborhood - Younger"
}
dt_mean.Cluster = dt_mean.Cluster.replace(clu_map)
dt_median.Cluster = dt_median.Cluster.replace(clu_map)

In [80]:
dt_mean_compile = dt_mean.copy(True)
for cl in dt_mean_compile.Cluster.unique():
    for m in dt_mean_compile.method.unique():
        for c in dt_mean.columns[3:]:
            dt_mean_compile.loc[(dt_mean_compile.method==m)&(dt_mean_compile.Cluster==cl), c]  = \
            str(dt_mean.loc[(dt_mean.method==m)&(dt_mean.Cluster==cl), c].values[0]) + \
            " " + "(" + str(dt_median.loc[(dt_median.method==m)&(dt_median.Cluster==cl), c].values[0]) + ")"
dt_mean_compile.to_excel("output/3clusters_descriptive.xlsx",index=False)

### Table 7 Std of Clusters

In [81]:
c_scale_mean = [c for c in dt_summary_fix.columns if c.endswith("scale_mean") ]
dt_scalemean = dt_summary_fix[[*['method', 'Cluster', 'n'],*c_scale_mean]].copy()

In [101]:
# dt_scalemean[dt_scalemean.Cluster==2].mean()
dt_scalemean[dt_scalemean.Cluster==1].std(), np.mean(dt_scalemean[dt_scalemean.Cluster==1].std().values[2:])

(Cluster                                0.000000
 n                                      9.689986
 polity_squared_scale_mean              0.095601
 PopYouthBulgeBy15_new_scale_mean       0.040451
 gdppc_ln_scale_mean                    0.026005
 nAC_perc_scale_mean                    0.094408
 imr_annual_mean_centered_scale_mean    0.039644
 sum_PDIS_EDIS_scale_mean               0.026833
 dtype: float64, 0.05382364570644602)

### Table 8- Case Membership

In [170]:
membership_list = []
for file in os.listdir("output"):
    if "3clust_" in file:
        method = file[7:-5]
        f_output = pd.ExcelFile(f"output/{file}")
        dt_membership = f_output.parse(sheet_name="membership")
        dt_pam = dt_membership[["country", "year", "pam_cluster"]].copy()
        dt_pam.columns = ["country", "year", "cluster_raw"]
        dt_pam.insert(0, "method", f"{method}_pam")
        dt_agnes = dt_membership[["country", "year", "agnes_cluster"]].copy()
        dt_agnes.columns = ["country", "year", "cluster_raw"]
        dt_agnes.insert(0, "method", f"{method}_agnes")
        dt_cl = pd.concat([dt_pam, dt_agnes])
        #
        membership_list.append(dt_cl)
dt_member = pd.concat(membership_list)
##
dt_summary_fix = pd.read_excel("output/3clusters_summary clusterfix 20220210.xlsx")
dt_member_fix = dt_summary_fix[["method", "Cluster","cluster_raw", "n"]]
dt_member = pd.merge(left = dt_member, right = dt_member_fix, on=["method", "cluster_raw"], how="left")
# dt_member.groupby(["method", "Cluster"]).count()

In [171]:
dt_member_pitf = dt_member[dt_member.method.str.contains("_pitf_")][["country", "year", "Cluster"]].reset_index(drop=True).copy()
cl1_list =[]
cl2_list = []
cl3_list = []
n_cl_list = []
for i in range(dt_member_pitf.shape[0]):
    c = dt_member_pitf.country[i]
    y = dt_member_pitf.year[i]
    cl1 = dt_member_pitf[(dt_member_pitf.country==c)&(dt_member_pitf.year==y)&(dt_member_pitf.Cluster==1)].shape[0]
    cl2 = dt_member_pitf[(dt_member_pitf.country==c)&(dt_member_pitf.year==y)&(dt_member_pitf.Cluster==2)].shape[0]
    cl3 = dt_member_pitf[(dt_member_pitf.country==c)&(dt_member_pitf.year==y)&(dt_member_pitf.Cluster==3)].shape[0]
    n_cl = [cl1,cl2,cl3].count(0) 
    if n_cl==2:
        n_cl_list.append(1)
    elif n_cl ==1:
        n_cl_list.append(2)
    elif n_cl ==0:
        n_cl_list.append(3)
    else:
        raise
    cl1_list.append(cl1)
    cl2_list.append(cl2)
    cl3_list.append(cl3)
dt_member_pitf["Anocratic - Younger - Less Developed"] = cl1_list
dt_member_pitf["Older - Wealthier - Moderate Discrimination"] = cl2_list
dt_member_pitf["Higher Discrimination - Worse Neighborhood - Younger"] = cl3_list
dt_member_pitf["Number of Clusters"] = n_cl_list
dt_member_pitf.drop(columns=["Cluster"]).drop_duplicates().to_excel("output/3clusters_membership clusterfix pitf.xlsx", index=False)

In [172]:
dt_member_pitf = dt_member[dt_member.method.str.contains("_prio_")][["country", "year", "Cluster"]].reset_index(drop=True).copy()
cl1_list =[]
cl2_list = []
cl3_list = []
n_cl_list = []
for i in range(dt_member_pitf.shape[0]):
    c = dt_member_pitf.country[i]
    y = dt_member_pitf.year[i]
    cl1 = dt_member_pitf[(dt_member_pitf.country==c)&(dt_member_pitf.year==y)&(dt_member_pitf.Cluster==1)].shape[0]
    cl2 = dt_member_pitf[(dt_member_pitf.country==c)&(dt_member_pitf.year==y)&(dt_member_pitf.Cluster==2)].shape[0]
    cl3 = dt_member_pitf[(dt_member_pitf.country==c)&(dt_member_pitf.year==y)&(dt_member_pitf.Cluster==3)].shape[0]
    n_cl = [cl1,cl2,cl3].count(0) 
    if n_cl==2:
        n_cl_list.append(1)
    elif n_cl ==1:
        n_cl_list.append(2)
    elif n_cl ==0:
        n_cl_list.append(3)
    else:
        raise
    cl1_list.append(cl1)
    cl2_list.append(cl2)
    cl3_list.append(cl3)
dt_member_pitf["Anocratic - Younger - Less Developed"] = cl1_list
dt_member_pitf["Older - Wealthier - Moderate Discrimination"] = cl2_list
dt_member_pitf["Higher Discrimination - Worse Neighborhood - Younger"] = cl3_list
dt_member_pitf["Number of Clusters"] = n_cl_list
dt_member_pitf.drop(columns=["Cluster"]).drop_duplicates().to_excel("output/3clusters_membership clusterfix prio.xlsx", index=False)

In [122]:
# country_year_prio_matched = []
# cases_prio = member_prio.country_year_prio.unique()
# cases_pitf = member_pitf.country_year_pitf.unique()
# for case in cases_pitf:
#     if case in cases_prio:
#         country_year_prio_matched.append(case)
#     else:
#         c = case.split("_")[0]
#         y = int(case.split("_")[1])
#         if c not in c_prio:
#             country_year_prio_matched.append(None)
#         else:
#             for gap in range(1,7):
#                 if gap >5:
#                     country_year_prio_matched.append(None)
#                     break
#                 else:
#                     if f"{c}_{y+gap}" in cases_prio:
#                         country_year_prio_matched.append(f"{c}_{y+gap}")
#                         break
#                     elif f"{c}_{y-gap}" in cases_prio:
#                         country_year_prio_matched.append(f"{c}_{y-gap}")
#                         break
